# Продвинутое машинное обучение: 
## Домашнее задание 3
Третье домашнее задание посвящено достаточно простой, но, надеюсь, интересной задаче, в которой потребуется творчески применить методы сэмплирования. Как и раньше, в качестве решения ожидается ссылка на jupyter-ноутбук на вашем github (или публичный, или с доступом для snikolenko); ссылку обязательно нужно прислать в виде сданного домашнего задания на портале Академии. Как всегда, любые комментарии, новые идеи и рассуждения на тему категорически приветствуются. 

В этом небольшом домашнем задании мы попробуем улучшить метод Шерлока Холмса.

Расшифруем несколько предложений простым частотным алгоритмом.

In [13]:
import re
from collections import Counter

In [75]:
with open('corpora/WarAndPeace.txt', 'r') as f:
    text_ru = f.read()
    
text_ru[:1000], len(text_ru)

('Annotation\n\n\n\t"Война и мир" – самый известный роман Льва Николаевича Толстого, как никакое другое произведение писателя, отражает глубину его мироощущения и философии.\tЭта книга из разряда вечных, потому что она обо всем – о жизни и смерти, о любви и чести, о мужестве и героизме, о славе и подвиге, о войне и мире.\tПервый том знакомит с высшим обществом России XIX века. Показаны взаимоотношения между родителями и детьми в семье Ростовых, сватовство у Болконских, интриги у Безуховых, вечера в салоне фрейлины А.П.Шерер, балы в Москве и Петербурге.\n\n\n\n\n\n* * *\n\n\n\nЛев Николаевич ТолстойЧАСТЬ ПЕРВАЯ\n\nЧАСТЬ BTОРАЯ\n\nЧАСТЬ ТРЕТЬЯ\n\n\n\n\n\n* * *\n\n\n\n\n\nЛев Николаевич Толстой\n\nВОЙНА И МИР\n\nТом 1\n\n\n\n\nЧАСТЬ ПЕРВАЯ\n\n\n\n\n\nI\n\n\n— Еh bien, mon prince. Gênes et Lucques ne sont plus que des apanages, des поместья, de la famille Buonaparte. Non, je vous préviens, que si vous ne me dites pas, que nous avons la guerre, si vous vous permettez encore de pallier toute

In [76]:
with open('corpora/WarAndPeaceEng.txt', 'r') as f:
    text_eng = f.read()
    
text_eng[:1000]

'\ufeffThe Project Gutenberg EBook of War and Peace, by Leo Tolstoy\n\nThis eBook is for the use of anyone anywhere at no cost and with almost\nno restrictions whatsoever.  You may copy it, give it away or re-use it\nunder the terms of the Project Gutenberg License included with this\neBook or online at www.gutenberg.org\n\n\nTitle: War and Peace\n\nAuthor: Leo Tolstoy\n\nTranslators: Louise and Aylmer Maude\n\nPosting Date: January 10, 2009 [EBook #2600]\n\nLast Updated: March 15, 2013\n\nLanguage: English\n\n\n*** START OF THIS PROJECT GUTENBERG EBOOK WAR AND PEACE ***\n\nAn Anonymous Volunteer, and David Widger\n\n\n\n\n\nWAR AND PEACE\n\nBy Leo Tolstoy/Tolstoi\n\nCONTENTS\n\nBOOK ONE: 1805\n\nCHAPTER I\n\nCHAPTER II\n\nCHAPTER III\n\nCHAPTER IV\n\nCHAPTER V\n\nCHAPTER VI\n\nCHAPTER VII\n\nCHAPTER VIII\n\nCHAPTER IX\n\nCHAPTER X\n\nCHAPTER XI\n\nCHAPTER XII\n\nCHAPTER XIII\n\nCHAPTER XIV\n\nCHAPTER XV\n\nCHAPTER XVI\n\nCHAPTER XVII\n\nCHAPTER XVIII\n\nCHAPTER XIX\n\nCHAPTER XX\n\nCH

In [78]:
def simple_text_preparation(text, regex):
    text = text.lower()
    text = text.replace('.', ' ')
    text = text.strip()
    text = re.sub(regex, '', text)
    return text

text_ru = simple_text_preparation(text_ru, r'[^а-я ]')
text_end = simple_text_preparation(text_eng, r'[^a-z ]')

counter_ru = Counter(text_ru)
counter_ru.most_common(10)

[(' ', 117256),
 ('о', 61282),
 ('а', 45209),
 ('е', 42519),
 ('и', 35838),
 ('н', 35119),
 ('т', 30619),
 ('с', 28128),
 ('л', 27277),
 ('в', 24824)]

In [88]:
def list_shuffle(l):
    l.remove(' ')
    before_ = l.copy()
    random.shuffle(l)
    after_ = l
    return dict(zip(before_, after_))


def text_shuffle(text, shuffle):
    new_text = list(text)
    for i, letter in enumerate(text):
        if letter in shuffle.keys():
            new_text[i] = shuffle[letter]
    return ''.join(new_text)
    
shuffle = list_shuffle(list(counter_ru))

new_text = text_shuffle(text_ru, shuffle)
    
new_text[:10], text_ru[:10]

('гшбуц с юс', 'война и ми')

Так мы производим замену букв в тексте с помощью случайной пересатновки алфавита. Попробуем восстановить текст по такой расстановке.

In [89]:
sample_size = 1000

train_begin = random.randint(0, len(text_ru) - sample_size)
test_begin = random.randint(0, len(text_ru) - sample_size)
train_text = text_ru[train_begin: train_begin + sample_size]
test_text = text_ru[test_begin: test_begin + sample_size]

train_text, test_text

('ел через очки то на него то на княгиню и зашевелился как будто он тоже хотел встать но опять раздумывал  что мне за дело что тут мсье пьер  вдруг сказала маленькая княгиня и хорошенькое лицо ее вдруг распустилось в слезливую гримасу   я тебе давно хотела сказать  за что ты ко мне так переменился что я тебе сделала ты едешь в армию ты меня не жалеешь  за что   только сказал князь андрей но в этом слове были и просьба и угроза и главное уверение в том что она сама раскается в своих словах но она торопливо продолжала ты обращаешься со мной как с больною или с ребенком  я вс вижу  разве ты такой был полгода назад  я прошу вас перестать  сказал князь андрей еще выразительнее пьер вс более и более приходивший в волнение во время этого разговора встал и подошел к княгине  он казалось не мог переносить вида слез и сам готов был заплакать  успокойтесь княгиня  вам это так кажется потому что я вас уверяю я сам испытал отчего потому что нет извините чужой тут лишний нет успокойтесь прощайтекняз

In [90]:
counter_train = Counter(train_text)
shuffle = list_shuffle(list(counter_train))

train_text_shuffled = text_shuffle(train_text, shuffle)
test_text_shuffled = text_shuffle(test_text, shuffle)
train_text_shuffled, test_text_shuffled

('нт чнрнд йчыа шй юл юнцй шй юл ыюгцаюя а длхнэнтатмг ылы ьбщшй йю шйкн вйшнт эмшлшп юй йогшп рлдщбзжэлт  чшй зюн дл щнтй чшй шбш змпн опнр  эщрбц мылдлтл злтнюпылг ыюгцаюг а вйрйхнюпыйн таий нн эщрбц рлмобмшатймп э мтндтаэбя цразлмб   г шньн щлэюй вйшнтл мылдлшп  дл чшй шж ый зюн шлы онрнзнюатмг чшй г шньн мщнтлтл шж нщнхп э лрзая шж знюг юн клтннхп  дл чшй   шйтпый мылдлт ыюгдп лющрнс юй э ушйз мтйэн ьжта а орймпьл а бцрйдл а цтлэюйн бэнрнюан э шйз чшй йюл млзл рлмылншмг э мэйав мтйэлв юй йюл шйрйотаэй орйщйтклтл шж йьрлелнхпмг мй зюйс ылы м ьйтпюйя ата м рньнюыйз  г эм эакб  рлдэн шж шлыйс ьжт ойтцйщл юлдлщ  г орйхб элм онрнмшлшп  мылдлт ыюгдп лющрнс нен эжрлдашнтпюнн опнр эм ьйтнн а ьйтнн оравйщаэхас э эйтюнюан эй эрнзг ушйцй рлдцйэйрл эмшлт а ойщйхнт ы ыюгцаюн  йю ылдлтймп юн зйц онрнюймашп эащл мтнд а млз цйшйэ ьжт длотлылшп  бмойыйсшнмп ыюгцаюг  элз ушй шлы ылкншмг ойшйзб чшй г элм бэнргя г млз аможшлт йшчнцй ойшйзб чшй юнш адэаюашн чбкйс шбш тахюас юнш бмойыйсшнмп орйелсшныюгд

In [109]:
counter_decode = Counter(test_text_shuffled)

encoded_freq_order = [letter[0] for letter in sorted(counter_decode.items(), key=lambda x: x[1], reverse=True)]
initial_freq_order = [letter[0] for letter in sorted(counter_ru.items(), key=lambda x: x[1], reverse=True)]

# make encoding
for i in range(max(len(encoded_freq_order) - len(initial_freq_order), 0)):
    initial_freq_order.append('*')
    
decoder = dict(zip(encoded_freq_order, initial_freq_order))

decoded_text = list(test_text_shuffled)
for i, letter in enumerate(decoded_text):
    decoded_text[i] = decoder[letter]

''.join(decoded_text)

'кнлметиз  ножиуе таномо щтевме л дашосе л мпиеря ыпниснонетл ысо веыетьисно иего ва йвеас ысо катесь л вараблгоись веыетьвлуон иоочхетеиь ножиуег доиуораа доиуораа чя кпгет роисон ыпниснпз ысо веуовафсо веиспдлто нрагз лйнакесь веитеюкавла есеул дро уосороа ов сеу гвомо итябет ос сонерлхажмпиерон  и чомог мачзсе  дройнпыет мотои кавлионе  мяияо гембн даракваг рзкп йеуотяшетлиь урпдя тобекаж  мреылу досзвпт донокьз л иег сровптиз идрене роисон нлкат дарняа рзкя инолш мпиер е аха кетьба ндаракл нлкватеиь агп сагвез дотоие уосорпц ов ва гом реиигосрась во иылсет вадрлзсатаг  няисратя чятл итябвя во н оскетавлл  дрлчень ряил  доитябетеиь уогевке л роисон ыпниснонет уеу доккеас йеког дарачлнез н метод амо мреылу ов ндарак пмекянет амо кнлюавлз л агп исевонлтоиь ниа наиатаа л наиатаа  ов йегаслт оклвоуоа карано ндаракл  эсо карано ивеыете чято ндаракл ве иараклва сож ыарся уосорез уейетеиь исоть исребвоц  е нос л дарабтл эсп ыарсп л ва сотьуо влыамо исребвомо ва чято во ни наиатаа л оюлнтав

 Получается, конечно, не очень. Можно лучше

## Проделаем то же самое для биграмм

In [121]:
bigrams = []
list_text_ru = list(text_ru)
for a, b in zip(list_text_ru[:-1], list_text_ru[1:]):
    if a != ' ' and b != ' ':
        bigrams.append(a + b)
        

In [125]:
bigrams_counter_ru = Counter(bigrams)

bigrams_encoded = []
list_text_ru = list(test_text_shuffled)
for a, b in zip(list_text_ru[:-1], list_text_ru[1:]):
    if a != ' ' and b != ' ':
        bigrams_encoded.append(a + b)
    
bigrams_encoded_counter = Counter(bigrams_encoded)

encoded_freq_order = [letter[0] for letter in sorted(bigrams_encoded_counter.items(), key=lambda x: x[1], reverse=True)]
initial_freq_order = [letter[0] for letter in sorted(bigrams_counter_ru.items(), key=lambda x: x[1], reverse=True)]
    
decoder = dict(zip(encoded_freq_order, initial_freq_order))

decoded_text = list(test_text_shuffled)
for i, letter in enumerate(decoded_text[:-1]):
    if decoded_text[i] != ' ' and decoded_text[i+1] != ' ':
        decoded_text[i] = decoder[decoded_text[i] + decoded_text[i+1]][0]

''.join(decoded_text)

'иаыжтсег  нсклол нвнирй уеиыжл а оеркрл а ропехж нстгбнпотма внй водтаугбнй пмлй рн ивбеш внй лоечап а расыислкгап водтаявннпэ мушлгтелзп нскломз лклнссон лклнссон чж швитт екгнпэ нстгбэжг внй втнтреонй влгтчоиой усрвг уатоочап влеелблаван чртща дей нкнсетн тю рты ооирй евамтт кш нпоедлдбиропеепэ  м чирмз пшбурл  дебаэндтт рдокм лавтмпол  маожй иьзеэ оасокррз лишб нтндвцытмтзп рсчеж огмалбс  сеосны лкежоит лпнвьег а пмз аетоисег ндеиол екгнпэ аилот оаусун лидж унсав ропер л адн отахан иоасоеа аикроелзп ргб дровнг лдокпл нкнссея тю рн лиц елевлкасеап ой ьсдртт реддоудонрз  сагасовж чима евакыж ой э ктотнавоа  ддяьияп хага  лкевамтелзп нмииуол а екгнпэ нстгбнпотт оты лваобеш наымз оасшуионг э жтоло срй сеосны тю иоасощ гжадтотт срй иарвавог а ргб гриопаиокзп тан тпаонон а тпаонон  тю нмоелит ветогнтн ласвнй иоасоеа  янй ласвнй аводтел чиой иоасоеа вл аасоетрн нсс еакыж нкнсенг оантелзп гндап гаезкобя  л нкш а оасыума ятб еактб а рн ндазнй впесрй гаезкоирй рн чиой ой тм тпаонон а сбионаа